### Logistic regression

 - Logistic rergression is a supervised learning algorithm
 - When the target variable is a quantitative variable, it is a regression problem, when it's a categorical one, it is a classification problem
 - A problem in the form of y = a + b*X
 - Ordinary Least Squares: the loss function minimizes the sum of the squared distance between the fitted line and the observations (residuals)
 - Evaluation with R^2 (% of variance explained by the features) or Root Square Mean Error (differences between the values predicted by a model and the values observed https://en.wikipedia.org/wiki/Root-mean-square_deviation)

 - Learn more: https://en.wikipedia.org/wiki/Linear_regression

In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import scale

# Import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [37]:
wines = load_wine()
all_features = pd.DataFrame(scale(wines['data']), columns = wines['feature_names'])
target = pd.DataFrame(all_features['alcohol'])
features = all_features.drop('alcohol',axis=1)

# Create arrays for the features and the response variable
y = target.values.reshape(-1)
X = features.values

display(y.shape, X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.35,
                                                    shuffle=False,
                                                    random_state=21)

(178,)

(178, 12)